In [8]:
!pip install nltk
!pip install benepar
!pip install spacy
%tensorflow_version 1.x
import benepar , spacy,nltk
benepar.download('benepar_en2')
from benepar.spacy_plugin import BeneparComponent

# Loading spaCy’s en model and adding benepar model to its pipeline
nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent('benepar_en2'))

[nltk_data] Downloading package benepar_en2 to /root/nltk_data...
[nltk_data]   Package benepar_en2 is already up-to-date!


In [10]:
class Node():
  def __init__(self):
    self.head=None
    self.parent=None
    self.tag = None
    self.count=0
    self.child=[]

def get_head_dictionary():
  head_dict={}
  head_dict['ADJP']={'dir':'r','tags':['RB','SBAR','RBS','RBR','FW','JJS','NP','JJR','ADJP','VBG','VBN','JJ','ADVP','NN','QP','NNS']}
  head_dict['ADVP']={'dir':'l','tags':['RB','RBR','RBS','FW','ADVP','TO','CD','JJR','JJ','IN','NP','JJS','NN']}
  head_dict['CONJP']={'dir':'l','tags':['CC','RB','IN']}
  head_dict['FRAG']={'dir':'l','tags':['NNPS','NNP','NNS','NN','NP','W','SBAR','PP','IN','ADJP','JJ','ADVP','RB']}
  head_dict['LST']={'dir':'l','tags':['LS',':']}
  head_dict['NP']={'dir':'l','tags':['NNPS','NNP','NNS','NN','NX','JJR','CD','JJ','JJS','RB','QP','NP']}
  head_dict['NX']=head_dict['NP']
  head_dict['PRT']={'dir':'l','tags':['RP']}
  head_dict['QP']={'dir':'r','tags':['$','IN','NNS','NN','JJ','RB','DT','CD','NCD','QP','JJR','JJS']}
  head_dict['RRC']={'dir':'l','tags':['VP','NP','ADVP','ADJP','PP']}
  head_dict['S']={'dir':'l','tags':['VP','S','SBAR','ADJP','UCP','NP']}
  head_dict['SBAR']={'dir':'r','tags':['S','SQ','SINV','SBAR','FRAG','IN','DT']}
  head_dict['SBARQ']={'dir':'r','tags':['SQ','S','SINV','SBARQ','FRAG']}
  head_dict['SINV']={'dir':'l','tags':['VB','VBD','VBP','VBZ','MD','VP','S','SINV','ADJP','NP']}
  head_dict['SQ']={'dir':'r','tags':['VB','VBD','VBP','VBZ','MD','VP','SQ']}
  head_dict['VP']={'dir':'l','tags':['VB','VBN','MD','VBZ','VBD','VBG','VBP','VP','AUX','ADJP','NN','NNS','NP']}
  head_dict['WHADJP']={'dir':'r','tags':['CC','WRB','JJ','ADJP']}
  head_dict['WHADVP']={'dir':'l','tags':['CC','WRB']}
  head_dict['WHNP']={'dir':'r','tags':['NN','WDT','WP','WP$','WHADJP','WHPP','WHNP']}
  head_dict['ADJP']['tags'].reverse()
  
  return head_dict

def generate_cp_input(cp):
  cp_input=[]
  word=''
  for i in range(len(cp)):
    if (cp[i]=='(' or cp[i]==')'):
      if (word!=''):
        cp_input.append(word)
        word=''
      cp_input.append(cp[i])
    elif cp[i]==' ' or cp[i] =='\n':
      if(word!=''):
        cp_input.append(word)
        word=''
      continue
    else:
      word+=cp[i]
  #print(cp_input)
  return cp_input
  return convert_to_universal_tags(cp_input)

head_dict=get_head_dictionary()

def convert_to_dp(cp):

  cp_input = generate_cp_input(cp)
  stack = []
  cur_word_no=1
  dp_relations={}

  AUX_VERBS=['is','were','can','could','dare','did','do','have','may','might','must','done','had','has','need','will','would','shall','should','outght']
  

  tags=set (['ROOT','LS', 'TO', 'VBN', 'WP', 'UH', 'VBG', 'JJ', 'VBZ', 'VBP', 'NN', 'DT', 
   'PRP', ':', 'WP$', 'NNPS', 'PRP$', 'WDT', '(', ')', '$', 'RB', 'RBR', 'RBS'
  , 'VBD', 'IN', 'FW', 'RP', 'JJR', 'JJS', 'PDT', 'MD', 'VB', 'WRB', 'NNP', 'EX', 'NNS', 'SYM', 'CC', 'CD', 'POS',
   'ADJP','ADVP','CONJP','FRAG','INTJ','LST','NAC','NP','NX','PP','WHPP','PRN','PRT',
   'QP','RRC','S','SBAR','SBARQ','SINV','SQ','VP','UCP','WHADJP','WHNP','WHADVP','X'])

  
  for string in cp_input:
    # if input contain ) , then pop nodes till ( is found and choose one node as head for all other nodes popped 
    if string==")" :
      v = []
      # poping nodes and storing in v
      while (stack[-1].head!="("):
        v.append(stack[-1])
        stack.pop()
      # last element popped will be parent phrase tag(NP ,VP ,S etc.) of all other nodes popped
      root = v[-1]
      # fin_head() returns index of element chosen as head according to Rules
      # updating root node ,'parent' denotes string chosen as head along with its index in input sentence as 'count'
      parent_index = find_head(v)
      root.parent = v[parent_index].parent
      root.count = v[parent_index].count
      #if v[parent_index].tag != None :
        #root.tag = v[parent_index].tag 

      for i in range(len(v)-1):
        root.child.append(v[i])
        if (i != parent_index):
          #print(root.tag , v[i].tag)
          dp_rel =find_dependency_relation(root,v[i])
          #relation=dp_rel+"("+root.parent+"-"+str(root.count)+" , "+v[i].parent+"-"+str(v[i].count)+")"
          relation=(dp_rel,root.parent,"-",root.count,",",v[i].parent,"-",v[i].count)
          dp_relations[v[i].count]=(relation)
      stack.pop()
      stack.append(root)

    else:
      node = Node()
      node.head = string
      node.parent = string
      if (string not in tags):
        node.count = cur_word_no
        cur_word_no+=1 
      else:
        node.tag=string
      if string in AUX_VERBS:
        stack[-1].head='AUX'
        stack[-1].parent='AUX'
        stack[-1].tag='AUX'
      stack.append(node)

  ROOT = stack[0]
  #relation="root( "+"ROOT-0 , "+ ROOT.parent+"-"+str(ROOT.count)+")"
  relation=('ROOT',ROOT.parent, '-' ,ROOT.count,',', ROOT.parent,"-",ROOT.count)
  dp_relations[ROOT.count]=relation
  dep_tree=[]
  for key in sorted(dp_relations):
    dep_tree.append(dp_relations[key])
    #print(dp_relations[key])
  return dep_tree


def find_dependency_relation(root,depend):
  PUNCT=['"',',','-LRB-','-RRB-','.',':','HYPH','``']
  ADV_TAGS=['ADV','ADVP','RB','RBR','RBS']
  AUX_VERBS=['is','were','can','could','dare','did','do','have','may','might','must','done','had','has','need','will','would','shall','should','outght']
  
  root_tag = root.tag
  child_tag = depend.tag
  root_pos =root.count
  child_pos = depend.count
  #print(child_tag,child_pos,root_tag,root_pos)
  # adverbial tag
  if(depend.parent in AUX_VERBS):
    return 'aux'
  if child_tag in ADV_TAGS:
    if root_tag in ['S','SBAR','SINV']:
      return 'advcl'
    if root_tag in ['NML','NP','QP']:
      return 'npadvmod'
    if root_tag=='VP':
      return 'advmod'
  if root_tag in ['VP','SINV','SQ','S','SBAR']:
    #print(child_pos,root_pos)
    if child_tag in ['NP','NML'] and child_pos < root_pos:
      return 'nsubj'
    if child_tag in ['NP','NML'] and child_pos > root_pos:
      return 'dobj'
    if child_tag in ['S','SQ','SBAR','SINV']:
      return 'relcl'
  if root_tag=='PP':
    if child_tag in ['NP','NML']:
      return 'pobj'
    if child_tag in ['IN']:
      return 'prep'
  if child_tag=='DT':
    return 'det'
  if child_tag=='PRP':
    return 'appos'
  if child_tag=='PRP$':
    return 'poss'
  if child_tag in PUNCT:
    return 'punct'
  if child_tag in ['ADJP','ADVP','WHADJP','WHADVP','JJ'] and root_tag=='NP':
    if child_pos < root_pos:
      return 'amod'
  if child_tag in ['PP','WHPP']:
    return 'prep'
  if child_tag in ['ADVP','SBAR'] and root_tag=='VP':
    if child_pos < root_pos:
      return 'advmod'
  if child_tag=='PRT' and root_tag=='VP':
    return 'prt'
  if child_tag=='VP' and root_tag in ['VP','SQ','SINV']:
    return 'vc'
  if root_tag in ['VP','S','SBAR','SBARQ','SINV','SQ']:
    if child_tag=='WHNP':
      return 'nsubj'
    return 'vmod'
  if root_tag in ['NP','NX','WHNP']:
    return 'nmod'
  if root_tag in ['ADJP','ADVP','WHADJP','WHADVP']:
    return 'amod'
  if root_tag in ['PP','WHPP']:
    return 'pmod'
  return 'dep'

def find_head(v):
  # considering each pos tag occurs one time in a phrase (chunk) otherwise take care of ordering and go for head initial
  PUNCT=['"',',','-LRB-','-RRB-','.',':','HYPH','``']
  tags_list = [v[i].tag for i in range(len(v)-1)]
  tags_list.reverse()
  #print(tags_list)
  len_tags=len(tags_list)-1
  root_tag=v[-1].head
  if(len(v)==2):
    return 0
  if(root_tag=='PP' or root_tag=='WHPP' ):
    prep_tags=['IN','RP']
    i=0
    for i in range(len(tags_list)):
      if tags_list[i] in PUNCT:
        continue
      if tags_list[i] in prep_tags:
        break
    return i+1
    #non puntuaction after prepposition
    
  if(root_tag=='PRN'):
    for i in range(len(tags_list)):
      if tags_list[i] in PUNCT:
        continue
      return i
    #first non punctuation
  if root_tag=='X' or root_tag=='UCP' or root_tag=='INTJ':
    return 0

  for t in head_dict[root_tag]['tags']:
    flag=0
    if head_dict[root_tag]['dir']=='r':
      tags_list.reverse()
      flag=1
    for i in range(len(tags_list)):
      #print('for root tag {} , head is {}'.format(root_tag,t))
      if(tags_list[i]==t):
        if flag==0:
          return len_tags-i
        else:
          return i
  return len(v)-2

def get_actual_cp(text):
  cp=list(nlp(text).sents)[0]._.parse_string
  return cp

def get_actual_dp(text):
  actual_tree=[]
  for token in nlp(text):
    s=token.dep_,token.head.text,'-',token.head.i+1,',',token.text,'-',token.i+1
    actual_tree.append(s)
  
  return actual_tree

def get_head_accuracy(actual_tree,predicted_tree):
  tp,total=0,0
  for true,pred in zip(actual_tree,predicted_tree):
    #print(true,pred)
    flag=0
    for t,p in zip(true[1:],pred[1:]):
      if t!=p:
        flag=1
        break
    if flag==0:
      tp+=1
    total+=1
  return tp/total

def get_relation_accuracy(actual_tree,predicted_tree):
  tp,total=0,0
  for true,pred in zip(actual_tree,predicted_tree):
    #print(true,pred)
    flag=0
    for t,p in zip(true[0:1],pred[0:1]):
      if t!=p:
        flag=1
        break
    if flag==0:
      tp+=1
    total+=1
  return tp/total


In [11]:
# give sentence for which you want to check dependency tree
sent="john runs quickly"
actual_cp=get_actual_cp(sent)
actual_dp=get_actual_dp(sent)
predicted_dp=convert_to_dp(actual_cp)

head_acc=get_head_accuracy(actual_dp,predicted_dp)
tag_acc=get_relation_accuracy(actual_dp,predicted_dp)
print("predicted dependency parse tree is:\n")
for relation in predicted_dp:
  print(relation)
print("head accuracy is {}".format(head_acc))
print("tag accuracy is {}".format(tag_acc))

predicted dependency parse tree is:

('nsubj', 'runs', '-', 2, ',', 'john', '-', 1)
('ROOT', 'runs', '-', 2, ',', 'runs', '-', 2)
('advmod', 'runs', '-', 2, ',', 'quickly', '-', 3)
head accuracy is 1.0
tag accuracy is 1.0
